<a href="https://colab.research.google.com/github/elliotgunn/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE

voting_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

In [0]:
import pandas as pd

In [0]:
# headers

column_headers = ['party','handicapped-infants','water-project-cost-sharing',
                 'adoption-of-the-budget-resolution','physician-fee-freeze',
                  'el-salvador-aid','religious-groups-in-schools',
                  'anti-satellite-test-ban','aid-to-nicaraguan-contras',
                 'mx-missile','immigration','synfuels-corporation-cutback',
                 'education-spending','superfund-right-to-sue','crime',
                 'duty-free-exports','export-administration-act-south-africa']

In [0]:
voting_data = pd.read_csv(voting_url, names=column_headers)

In [0]:
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


NaNs could mean absent or abstaining

In [0]:
# fill ?s with NaN
import numpy as np

voting_data = voting_data.replace('?', np.NaN)

In [0]:
# change y = 1.0
# change n = 0.0

voting_data = voting_data.replace({'y':1, 'n':0})

In [0]:
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
# split into two datasets

rep = voting_data[voting_data['party'] == 'republican']
dem = voting_data[voting_data['party'] == 'democrat']

# 1-sample t-test

In [0]:
dem['handicapped-infants'].value_counts()

1.0    156
0.0    102
Name: handicapped-infants, dtype: int64

In [0]:
dem['handicapped-infants'].isnull().sum()

9

In [0]:
from scipy.stats import ttest_1samp

ttest_1samp(dem['handicapped-infants'], 0, nan_policy='omit')

# this is questioning proportion of dems voting yes compared to null of no ('0')
# can i say there is no dem support?
# p value says nope, you can't say that

Ttest_1sampResult(statistic=19.825711173357988, pvalue=1.0391992873567661e-53)

Null: there is 0 dem support for this bill
alt hypothesis: that there is non-0 dem support
p-value threshold (confidence level): 95% confidence level (0.05 p value threshold)

Given the result of the above test I would reject the null that dem support is 0 at the 95% significance level. 

# 2-sample t-test (for means) example

In [0]:
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [0]:
print("Democrat Support: ", dem['export-administration-act-south-africa'].mean())
print("Republican Support: ", rep['export-administration-act-south-africa'].mean())

Democrat Support:  0.9351351351351351
Republican Support:  0.6575342465753424


In [0]:
ttest_ind(dem['export-administration-act-south-africa'],
               rep['export-administration-act-south-africa'],
               nan_policy='omit')

Ttest_indResult(statistic=6.849454815841208, pvalue=3.652674361672226e-11)

is rate of support in dems same as rep?

Null: the mean of dem support == mean of repub support (two parties support the bill at the same rate)

alt: means are different (not the same level of support)

T stat goes up, p value goes down (inversely correlated)


# Assignment
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

write out hypothesis, write out conclusions

## Dems support more than repubs

In [0]:
print("Democrat Support: ", dem['aid-to-nicaraguan-contras'].mean())
print("Republican Support: ", rep['aid-to-nicaraguan-contras'].mean())

Democrat Support:  0.8288973384030418
Republican Support:  0.15286624203821655


In [0]:
ttest_ind(dem['aid-to-nicaraguan-contras'],
               rep['aid-to-nicaraguan-contras'],
               nan_policy='omit')

Ttest_indResult(statistic=18.052093200819733, pvalue=2.82471841372357e-54)

Null: the mean of dem support == mean of repub support (two parties support the bill at the same rate)

alt: means are different (not the same level of support)

Given the result of the above test I would reject the null that both parties support the same issue at the same rate. 

## Repubs support more than dems

In [0]:
voting_data.head()

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
print("Democrat Support: ", dem['religious-groups-in-schools'].mean())
print("Republican Support: ", rep['religious-groups-in-schools'].mean())

Democrat Support:  0.47674418604651164
Republican Support:  0.8975903614457831


In [0]:
ttest_ind(dem['religious-groups-in-schools'],
               rep['religious-groups-in-schools'],
               nan_policy='omit')

Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20)

Null: the mean of dem support == mean of repub support (two parties support the bill at the same rate)

alt: means are different (not the same level of support)

Given the result of the above test I would reject the null that both parties support the same issue at the same rate. 


## difference between republicans and democrats has p > 0.1

In [0]:
print("Democrat Support: ", dem['water-project-cost-sharing'].mean())
print("Republican Support: ", rep['water-project-cost-sharing'].mean())

Democrat Support:  0.502092050209205
Republican Support:  0.5067567567567568


In [0]:
ttest_ind(dem['water-project-cost-sharing'],
               rep['water-project-cost-sharing'],
               nan_policy='omit')

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

Null: the mean of dem support == mean of repub support (two parties support the bill at the same rate)

alt: means are different (not the same level of support)

Given the result of the above test I would fail to reject the null that both parties support the same issue at the same rate. 